In [44]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date, timedelta

In [45]:
staffing_df = pd.read_excel('../../Provided_Data/4 District Platoon Staffing Exceptions 2020 to 2022.xlsx')
print(staffing_df.shape)
staffing_df.head()


(28374, 6)


,Platoon,Officer Badge #,Exception Date,Exception Start Time,# of Hours,Exception Type
0,4 District B Platoon,34260.0,20200227.0,07:00:00,12.0,Vacation
1,4 District B Platoon,34260.0,20200220.0,07:00:00,10.0,Training
2,4 District B Platoon,34260.0,20200219.0,07:00:00,10.0,Training
3,4 District B Platoon,34260.0,20200307.0,07:00:00,12.0,Sick
4,4 District B Platoon,34260.0,20200228.0,07:00:00,12.0,Vacation


In [46]:
# Convert the column to datetime format
staffing_df['date'] = pd.to_datetime(staffing_df['Exception Date'], format='%Y%m%d')
# Convert the datetime column to the desired format
staffing_df['date'] = staffing_df['date'].dt.strftime('%Y-%m-%d')
staffing_df

,Platoon,Officer Badge #,Exception Date,Exception Start Time,# of Hours,Exception Type,date
0,4 District B Platoon,34260.0,20200227.0,07:00:00,12.0,Vacation,2020-02-27
1,4 District B Platoon,34260.0,20200220.0,07:00:00,10.0,Training,2020-02-20
2,4 District B Platoon,34260.0,20200219.0,07:00:00,10.0,Training,2020-02-19
3,4 District B Platoon,34260.0,20200307.0,07:00:00,12.0,Sick,2020-03-07
4,4 District B Platoon,34260.0,20200228.0,07:00:00,12.0,Vacation,2020-02-28
...,...,...,...,...,...,...,...
28369,4 District C Platoon,38160.0,20221222.0,18:00:00,12.0,Sick,2022-12-22
28370,4 District D Platoon,37350.0,20221222.0,07:00:00,12.0,Vacation,2022-12-22
28371,4 District D Platoon,37350.0,20221230.0,07:00:00,6.0,Vacation,2022-12-30
28372,4 District D Platoon,37350.0,20221231.0,19:00:00,12.0,Vacation,2022-12-31


In [47]:
staffing_df['date'] = pd.to_datetime(staffing_df['date'])
staffing_df.dtypes

Platoon                         object
Officer Badge #                float64
Exception Date                 float64
Exception Start Time            object
# of Hours                     float64
Exception Type                  object
date                    datetime64[ns]
dtype: object

In [48]:
def get_date_after_n_days(date, n):
    # Convert string to datetime
    date_obj = datetime.strptime(date, '%Y-%m-%d')
    # Add n days
    new_date_obj = date_obj + timedelta(days=n)
    # Convert back to string in 'Y-M-D' format
    new_string_date = new_date_obj.strftime('%Y-%m-%d')
    return new_string_date

In [49]:
start_date, end_date = '2022-06-01', '2022-12-31'
total_officer = staffing_df['Officer Badge #'].nunique()

# staffing_future = staffing_df.loc[(staffing_df['date']>=start_date) & (staffing_df['date']<=get_date_after_n_days(start_date, 30))]
# staffing_future

In [50]:
# Get number of occurances each day
staffing_exceptions_df = staffing_df.groupby('date').size().reset_index(name='num_of_exceptions')
# Convert 'date' column to datetime type
staffing_exceptions_df['date'] = pd.to_datetime(staffing_exceptions_df['date'])
staffing_exceptions_df['expected_officers'] = total_officer-staffing_exceptions_df['num_of_exceptions']
staffing_exceptions_df

,date,num_of_exceptions,expected_officers
0,2020-01-01,26,254
1,2020-01-02,18,262
2,2020-01-03,19,261
3,2020-01-04,10,270
4,2020-01-05,16,264
...,...,...,...
1091,2022-12-27,27,253
1092,2022-12-28,27,253
1093,2022-12-29,32,248
1094,2022-12-30,35,245


In [51]:
# Read crime forecast data
crime_forecast_df = pd.read_csv('Results/crime_forecast.csv')
crime_forecast_df.rename(columns={'y_pred': 'num_of_crimes_predicted'}, inplace=True)
# Convert 'date' column to datetime type
crime_forecast_df['date'] = pd.to_datetime(crime_forecast_df['date'])
print(crime_forecast_df.shape)
crime_forecast_df.head()

(761, 14)


,Unnamed: 0,date,previous_day_crimes,last_7_days_crimes,last_30_days_crimes,dayofweek,month,year,dayofmonth,weekofyear,is_weekend,season,num_of_crimes,num_of_crimes_predicted
0,0,2021-06-01,76.0,569.0,2494.0,1,6,2021,1,22,0,2,167,178.354840
1,1,2021-06-02,167.0,644.0,2590.0,2,6,2021,2,22,0,2,100,85.391076
2,2,2021-06-03,100.0,667.0,2604.0,3,6,2021,3,22,0,2,72,87.940413
3,3,2021-06-04,72.0,657.0,2589.0,4,6,2021,4,22,0,2,91,91.893445
4,4,2021-06-05,91.0,660.0,2596.0,5,6,2021,5,22,1,2,72,88.774504


In [52]:
# Read service calls forecast data
service_calls_forecast_df = pd.read_csv('Results/service_calls_forecast.csv')
service_calls_forecast_df.rename(columns={'y_pred': 'num_of_calls_predicted'}, inplace=True)
# Convert 'date' column to datetime type
service_calls_forecast_df['date'] = pd.to_datetime(service_calls_forecast_df['date'])
print(service_calls_forecast_df.shape)
service_calls_forecast_df.head()

(214, 14)


,Unnamed: 0,date,previous_day_calls,last_7_days_calls,last_30_days_calls,dayofweek,month,year,dayofmonth,weekofyear,is_weekend,season,num_of_calls,num_of_calls_predicted
0,0,2022-06-01,472.0,2611.0,12945.0,2,6,2022,1,22,0,2,487,459.094241
1,1,2022-06-02,487.0,2687.0,13083.0,3,6,2022,2,22,0,2,442,470.840328
2,2,2022-06-03,442.0,2682.0,13149.0,4,6,2022,3,22,0,2,473,454.237026
3,3,2022-06-04,473.0,2749.0,13233.0,5,6,2022,4,22,1,2,453,416.298850
4,4,2022-06-05,453.0,2780.0,13242.0,6,6,2022,5,22,1,2,425,406.442894


In [53]:
staff_crime_calls_merged = staffing_exceptions_df.merge(crime_forecast_df[['date', 'num_of_crimes_predicted']], on='date', how='inner').merge(service_calls_forecast_df[['date', 'num_of_calls_predicted']], on='date', how='inner')
staff_crime_calls_merged

,date,num_of_exceptions,expected_officers,num_of_crimes_predicted,num_of_calls_predicted
0,2022-06-01,29,251,193.939090,459.094241
1,2022-06-02,27,253,115.519179,470.840328
2,2022-06-03,25,255,103.792778,454.237026
3,2022-06-04,25,255,99.690775,416.298850
4,2022-06-05,25,255,95.237076,406.442894
...,...,...,...,...,...
209,2022-12-27,27,253,89.136301,369.885594
210,2022-12-28,27,253,92.571424,372.816549
211,2022-12-29,32,248,91.237752,396.377441
212,2022-12-30,35,245,87.297976,387.622820


# Ranking staff-crimes-calls

In [56]:
staff_crime_calls_merged['expected_officers_rank'] = staff_crime_calls_merged['expected_officers'].rank(pct=True)
staff_crime_calls_merged['num_of_crimes_predicted_rank'] = staff_crime_calls_merged['num_of_crimes_predicted'].rank(pct=True)
staff_crime_calls_merged['num_of_calls_predicted_rank'] = staff_crime_calls_merged['num_of_calls_predicted'].rank(pct=True)

In [60]:
def get_rank_bucket(df, column):
    conditions = [
        (df[column+'_rank'] <= 0.33),
        (df[column+'_rank'] > 0.33) & (df[column+'_rank'] <= 0.66),
        (df[column+'_rank'] > 0.66)
    ]
    values = ['Low', 'Medium', 'High']

    df[column+'_bucket'] = np.select(conditions, values)

    return df


In [62]:
get_rank_bucket(staff_crime_calls_merged, 'expected_officers')
get_rank_bucket(staff_crime_calls_merged, 'num_of_crimes_predicted')
get_rank_bucket(staff_crime_calls_merged, 'num_of_calls_predicted')
staff_crime_calls_merged

,date,num_of_exceptions,expected_officers,num_of_crimes_predicted,num_of_calls_predicted,expected_officers_rank,num_of_crimes_predicted_rank,num_of_calls_predicted_rank,expected_officers_bucket,num_of_crimes_predicted_bucket,num_of_calls_predicted_bucket
0,2022-06-01,29,251,193.939090,459.094241,0.581776,0.985981,0.920561,Medium,High,High
1,2022-06-02,27,253,115.519179,470.840328,0.686916,0.948598,0.948598,High,High,High
2,2022-06-03,25,255,103.792778,454.237026,0.792056,0.523364,0.911215,High,Medium,High
3,2022-06-04,25,255,99.690775,416.298850,0.792056,0.266355,0.429907,High,Low,Medium
4,2022-06-05,25,255,95.237076,406.442894,0.792056,0.084112,0.261682,High,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...
209,2022-12-27,27,253,89.136301,369.885594,0.686916,0.009346,0.070093,High,Low,Low
210,2022-12-28,27,253,92.571424,372.816549,0.686916,0.023364,0.098131,High,Low,Low
211,2022-12-29,32,248,91.237752,396.377441,0.401869,0.014019,0.205607,Medium,Low,Low
212,2022-12-30,35,245,87.297976,387.622820,0.228972,0.004673,0.140187,Low,Low,Low
